## Group 8_Lab5

Integrants: 

- Giafranco Soria (20163509) 
- Andrea Clavo (20176040)
- Sandra Martinez (20173026)

In [40]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
Penn = pd.read_csv("C:/Users/andre/OneDrive/Documents/GitHub/ECO224/Labs/data/penn_jae.dat" , sep='\s', engine='python')
n = Penn.shape[0]
#numero de covariables 
p_1 = Penn.shape[1]
Penn = Penn[ (Penn['tg'] == 4) | (Penn['tg'] == 0) ]

In [42]:
Penn.columns
Penn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5099 entries, 0 to 13911
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   abdt         5099 non-null   int64  
 1   tg           5099 non-null   int64  
 2   inuidur1     5099 non-null   int64  
 3   inuidur2     5099 non-null   int64  
 4   female       5099 non-null   int64  
 5   black        5099 non-null   int64  
 6   hispanic     5099 non-null   int64  
 7   othrace      5099 non-null   int64  
 8   dep          5099 non-null   int64  
 9   q1           5099 non-null   int64  
 10  q2           5099 non-null   int64  
 11  q3           5099 non-null   int64  
 12  q4           5099 non-null   int64  
 13  Unnamed: 13  5099 non-null   int64  
 14  q5           5099 non-null   int64  
 15  q6           5099 non-null   int64  
 16  recall       5099 non-null   int64  
 17  agelt35      5099 non-null   int64  
 18  agegt54      5099 non-null   int64  
 19  durab

In [43]:
# Dependent variable
Penn['T4'] = (Penn[['tg']]==4).astype(int)

# Create category variable
Penn['dep'] = Penn['dep'].astype( 'category' )
Penn.head()


,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,0,0,1,0,NaN,0
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
11,10607,4,9,9,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,NaN,1
12,10831,0,27,27,0,0,0,0,1,0,...,0,0,1,1,0,1,0,0,NaN,0


In [44]:
Penn['dep'].unique()

[2, 0, 1]
Categories (3, int64): [2, 0, 1]

In [45]:
Penn.columns

Index(['abdt', 'tg', 'inuidur1', 'inuidur2', 'female', 'black', 'hispanic',
       'othrace', 'dep', 'q1', 'q2', 'q3', 'q4', 'Unnamed: 13', 'q5', 'q6',
       'recall', 'agelt35', 'agegt54', 'durable', 'nondurable', 'lusd', 'husd',
       'muld', 'T4'],
      dtype='object')

In [46]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import numpy as np

In [47]:
Penn["inuidur"] = np.log (Penn["inuidur1"])  

In [48]:
model =  "inuidur1~T4+ (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)"
model_results = smf.ols( model , data = Penn ).fit().get_robustcov_results(cov_type = "HC1")

In [49]:
print(model_results.summary())
print( "Number of regressors in the basic model:",len(model_results.params), '\n')

                            OLS Regression Results                            
Dep. Variable:               inuidur1   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     9.171
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           8.01e-23
Time:                        15:21:08   Log-Likelihood:                -19176.
No. Observations:                5099   AIC:                         3.839e+04
Df Residuals:                    5082   BIC:                         3.850e+04
Df Model:                          16                                         
Covariance Type:                  HC1                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      10.9325      0.427     25.576      

In [50]:
coeff1 =model_results.summary2().tables[1]['Coef.']['T4']
se1 =model_results.summary2().tables[1]['Std.Err.']['T4']

coeff2 =model_results.summary2().tables[1]['Coef.']['female']
se2 =model_results.summary2().tables[1]['Std.Err.']['female']

coeff3 =model_results.summary2().tables[1]['Coef.']['black']
se3 =model_results.summary2().tables[1]['Std.Err.']['black']

In [80]:
coeff1

-0.7327816888029653

#### To compare our results derived from bootstrapping, we extracted the coefficients estimated by OLS with the initial data.

In [51]:
df=pd.DataFrame({"Name":['T4','female','black'], 
                 "coeff":[coeff1,coeff2,coeff3],
                 "s.e":[se1,se2,se3]}) 
df

,Name,coeff,s.e
0,T4,-0.732782,0.306175
1,female,1.530674,0.306565
2,black,-1.318531,0.496567


## Now, we proceed to Bootstrapping

a) First, we create the function get_indices which would be the equivalent of the given function "sample" in R. That is, we randomly choose the indices to use over the number of observations with replacement. 

In [52]:
def get_indices(data,num_samples):
    return  np.random.choice(data.index, num_samples, replace=True)


b) We generate the selected model

In [62]:
def get_estimates(data,index):
    data_1 = data.loc[index]

    model = "inuidur1~T4+ (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)"

    model_results = smf.ols( model , data = data_1 ).fit().get_robustcov_results(cov_type = "HC1")

    T4_coeff = model_results.summary2().tables[1]['Coef.']['T4']
    female_coeff = model_results.summary2().tables[1]['Coef.']['female']
    black_coeff = model_results.summary2().tables[1]['Coef.']['black']

    return [T4_coeff,female_coeff, black_coeff]


c) We create the boot function, which is equivalent to the "boot" function of R. In it we select the data we want, the functional form we will use and the number of replicas. Here we calculate the average of the randomly selected coefficients and standard errors. 

In [63]:
def boot(data,func,R):
    T4 = []
    female = []
    black = []
    for i in range(R):
        T4.append(func(data,get_indices(data,5099))[0])
        female.append(func(data,get_indices(data,5099))[1]) 
        black.append(func(data,get_indices(data,5099))[2]) 
    T4_statistics = {'estimated_value':np.mean(T4),'std_error':np.std(T4)}   
    female_statistices = {'estimated_value':np.mean(female),'std_error':np.std(female)}   
    black_statistices = {'estimated_value':np.mean(black),'std_error':np.std(black)} 
    return {'T4':T4_statistics,'female_statistices':female_statistices,'black_statistices':black_statistices}


d) Thus, we now have our boostrap function. That is, from the Penn data I will select with the form of estimates and we do this Bootstraping exercise 1000 times. 


In [64]:
results =boot(Penn,get_estimates,1000)

In [65]:
results

{'T4': {'estimated_value': -0.7363693457860303,
  'std_error': 0.29606634020584593},
 'female_statistices': {'estimated_value': 1.5189116477691196,
  'std_error': 0.30663167608028963},
 'black_statistices': {'estimated_value': -1.3340835778996443,
  'std_error': 0.486111104968979}}

In [66]:
coeff4 =results['T4']['estimated_value']
se4 =results['T4']['std_error']

coeff5 =results['female_statistices']['estimated_value']
se5 =results['female_statistices']['std_error']

coeff6 =results['black_statistices']['estimated_value']
se6 =results['black_statistices']['std_error']

#### We extract the coefficients of interest from the Bootstraping data. 

In [77]:
data={"T4":[coeff1,se1], 
                 "T4_bs":[coeff4,se4],
                 "female":[coeff2,se3],
                 "female_bs":[coeff5,se5],
                 "black":[coeff3,se3],
                 "black_bs":[coeff6,se6]}

In [78]:
df=pd.DataFrame(data, index=['coeff', 'se']) 

In [79]:
df

,T4,T4_bs,female,female_bs,black,black_bs
coeff,-0.732782,-0.736369,1.530674,1.518912,-1.318531,-1.334084
se,0.306175,0.296066,0.496567,0.306632,0.496567,0.486111


Finally, we find that some changes in the coefficients of interest when we estimate them in the traditional way than when we perform Bootstraping. The difference is not so large since the number of replicates has been quite large (1000). 